In [6]:
from docplex.mp.model import Model
from docplex import *
# from qiskit import BasicAer
# from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import CplexOptimizer, MinimumEigenOptimizer
from qiskit_optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer
from qiskit_optimization import QuadraticProgram

from qiskit_optimization.converters import InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty
from qiskit_optimization.translators import from_docplex_mp
from brute_force_bpp import *

## Formulation

The **Bin Packing Problem (BPP)** is a classic optimization problem where we are given a set of items with specific weights and bins with a fixed capacity. The goal is to pack all items into the minimum number of bins without exceeding the capacity of any bin.

### Problem Definition

- Given $n$ items with weights $w_1, w_2, \ldots, w_n$ and bin capacity $C$.
- Minimize the number of bins used.

### ILP Formulation

**Decision Variables:**
- $x_{ij}$: 1 if item $i$ is in bin $j$, 0 otherwise.
- $y_j$: 1 if bin $j$ is used, 0 otherwise.

**Objective Function:**
$$
\text{Minimize } \sum_{j=1}^{m} y_j
$$
where $m$ is an upper bound on the number of bins.

**Constraints:**
1. Each item must be assigned to exactly one bin:
   $$
   \sum_{j=1}^{m} x_{ij} = 1, \quad \forall i \in \{1, 2, \ldots, n\}
   $$
2. The total weight in each bin cannot exceed its capacity:
   $$
   \sum_{i=1}^{n} w_i x_{ij} \leq C y_j, \quad \forall j \in \{1, 2, \ldots, m\}
   $$
3. Binary constraints:
   $$
   x_{ij} \in \{0, 1\}, \quad y_j \in \{0, 1\}
   $$

### Explanation of Terms
- **Objective Function**: Represents the total number of bins used.
- **Constraint 1**: Ensures each item is placed in exactly one bin.
- **Constraint 2**: Ensures the weight in each bin does not exceed $C$.
- **Binary Constraints**: Ensure $x_{ij}$ and $y_j$ are binary variables.


In [2]:
# Input Data
weights = [5, 2, 7, 3, 1]  # Weights of each item
n = len(weights)  # Number of items
C = max(weights) + 1  # Bin capacity
m = n  # Maximum possible number of bins (one item per bin at most)

# Initialize model
model = Model(name="Bin Packing Problem")

# Decision Variables
# x[i, j] = 1 if item i is placed in bin j, 0 otherwise
x = {(i, j): model.binary_var(name=f"x_{i}_{j}") for i in range(n) for j in range(m)}

# y[j] = 1 if bin j is used, 0 otherwise
y = {j: model.binary_var(name=f"y_{j}") for j in range(m)}

In [3]:
# Constraints

# 1. Each item must be assigned to exactly one bin
for i in range(n):
    model.add_constraint(model.sum(x[i, j] for j in range(m)) == 1, ctname=f"assign_item_{i}")

# 2. Total weight in each bin cannot exceed bin capacity
for j in range(m):
    model.add_constraint(model.sum(weights[i] * x[i, j] for i in range(n)) <= C * y[j], ctname=f"capacity_bin_{j}")

# Objective: Minimize the number of bins used
model.minimize(model.sum(y[j] for j in range(m)))



# Solve the model
# solution = model.solve()

In [4]:
model.lp_string

'\\ This file has been generated by DOcplex\n\\ ENCODING=ISO-8859-1\n\\Problem name: Bin Packing Problem\n\nMinimize\n obj: y_0 + y_1 + y_2 + y_3 + y_4\nSubject To\n assign_item_0: x_0_0 + x_0_1 + x_0_2 + x_0_3 + x_0_4 = 1\n assign_item_1: x_1_0 + x_1_1 + x_1_2 + x_1_3 + x_1_4 = 1\n assign_item_2: x_2_0 + x_2_1 + x_2_2 + x_2_3 + x_2_4 = 1\n assign_item_3: x_3_0 + x_3_1 + x_3_2 + x_3_3 + x_3_4 = 1\n assign_item_4: x_4_0 + x_4_1 + x_4_2 + x_4_3 + x_4_4 = 1\n capacity_bin_0: 5 x_0_0 + 2 x_1_0 + 7 x_2_0 + 3 x_3_0 + x_4_0 - 8 y_0 <= 0\n capacity_bin_1: 5 x_0_1 + 2 x_1_1 + 7 x_2_1 + 3 x_3_1 + x_4_1 - 8 y_1 <= 0\n capacity_bin_2: 5 x_0_2 + 2 x_1_2 + 7 x_2_2 + 3 x_3_2 + x_4_2 - 8 y_2 <= 0\n capacity_bin_3: 5 x_0_3 + 2 x_1_3 + 7 x_2_3 + 3 x_3_3 + x_4_3 - 8 y_3 <= 0\n capacity_bin_4: 5 x_0_4 + 2 x_1_4 + 7 x_2_4 + 3 x_3_4 + x_4_4 - 8 y_4 <= 0\n\nBounds\n 0 <= x_0_0 <= 1\n 0 <= x_0_1 <= 1\n 0 <= x_0_2 <= 1\n 0 <= x_0_3 <= 1\n 0 <= x_0_4 <= 1\n 0 <= x_1_0 <= 1\n 0 <= x_1_1 <= 1\n 0 <= x_1_2 <= 1\n 

In [ ]:
mod = from_docplex_mp(model)
# Solving Quadratic Program using CPLEX
cplex = CplexOptimizer()
result = cplex.solve(mod)
print(result)
# ineq2eq = InequalityToEquality()
# qp_eq = ineq2eq.convert(mod)
# print(qp_eq.export_as_lp_string())
# print(f"The number of variables is {qp_eq.get_num_vars()}")

In [57]:
import numpy as np
import matplotlib.pyplot as plt
def data_bins(results, wj, n, m, l=0, simplify=False):
    """save the results on a dictionary with the three items, bins, items, and index.
    results (cplex.solve): results of the optimization
    wj: (array (1,m): weights of the items
    n: (int) number of items
    m: (int) number of bins
    """
    if simplify:
        bins = np.ones((m,))
        if m-l > 0: 
            bins[m-l-1:m] = results[:m-l]
        items = np.zeros((m,n))
        items[:,1:] =  results[m-l:(m-1)*n+m-l].reshape(m,n-1)
        items[0,0] = 1
        items = items.reshape(m,n) * wj
        return {"bins":bins, "items":items,"index":np.arange(m)}
    else:        
        return {"bins":results[:m], "items":results[m:m+m*n].reshape(m,n) * wj, "index":np.arange(m)}
def plot_bins(results, wj, n, m, l=0,simplify=False):
    """plot in a bar diagram the results of an optimization bin packing problem"""
    res = data_bins(results.x, wj, n, m, l, simplify)
    plt.figure()
    ind = res["index"]
    plt.bar(ind, res["items"][:,0], label=f"item {0}")
    suma = bottom=res["items"][:,0]
    for j in range(1,n):
        plt.bar(ind, res["items"][:,j], bottom=suma, label=f"item {j}")
        suma += res["items"][:,j]
    # plt.hlines(Q,0-0.5,m-0.5,linestyle="--", color="r",label="Max W")
    plt.xticks(ind)
    plt.xlabel("Bin")
    plt.ylabel("Weight")
    plt.legend()

In [56]:
plot_bins(result, weights, n, m)

NameError: name 'result' is not defined

In [36]:

int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
print(qp_eq_bin.export_as_lp_string())
print(f"The number of variables is {qp_eq_bin.get_num_vars()}")

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Bin Packing Problem

Minimize
 obj: y_0 + y_1 + y_2 + y_3 + y_4
Subject To
 assign_item_0: x_0_0 + x_0_1 + x_0_2 + x_0_3 + x_0_4 = 1
 assign_item_1: x_1_0 + x_1_1 + x_1_2 + x_1_3 + x_1_4 = 1
 assign_item_2: x_2_0 + x_2_1 + x_2_2 + x_2_3 + x_2_4 = 1
 assign_item_3: x_3_0 + x_3_1 + x_3_2 + x_3_3 + x_3_4 = 1
 assign_item_4: x_4_0 + x_4_1 + x_4_2 + x_4_3 + x_4_4 = 1
 capacity_bin_0: 5 x_0_0 + 2 x_1_0 + 7 x_2_0 + 3 x_3_0 + x_4_0 - 8 y_0
                 + capacity_bin_0@int_slack@0 + 2 capacity_bin_0@int_slack@1
                 + 4 capacity_bin_0@int_slack@2 + capacity_bin_0@int_slack@3 = 
                 0
 capacity_bin_1: 5 x_0_1 + 2 x_1_1 + 7 x_2_1 + 3 x_3_1 + x_4_1 - 8 y_1
                 + capacity_bin_1@int_slack@0 + 2 capacity_bin_1@int_slack@1
                 + 4 capacity_bin_1@int_slack@2 + capacity_bin_1@int_slack@3 = 
                 0
 capacity_bin_2: 5 x_0_2 + 2 x_1_2 + 7 x_2_2 + 3 x_3_2 + x_4_2

In [37]:
lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.convert(qp_eq_bin)
print(f"The number of variables is {qp_eq_bin.get_num_vars()}")
print(qubo.export_as_lp_string())

The number of variables is 50
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Bin Packing Problem

Minimize
 obj: - 12 x_0_0 - 12 x_0_1 - 12 x_0_2 - 12 x_0_3 - 12 x_0_4 - 12 x_1_0
      - 12 x_1_1 - 12 x_1_2 - 12 x_1_3 - 12 x_1_4 - 12 x_2_0 - 12 x_2_1
      - 12 x_2_2 - 12 x_2_3 - 12 x_2_4 - 12 x_3_0 - 12 x_3_1 - 12 x_3_2
      - 12 x_3_3 - 12 x_3_4 - 12 x_4_0 - 12 x_4_1 - 12 x_4_2 - 12 x_4_3
      - 12 x_4_4 + y_0 + y_1 + y_2 + y_3 + y_4 + [ 312 x_0_0^2 + 24 x_0_0*x_0_1
      + 24 x_0_0*x_0_2 + 24 x_0_0*x_0_3 + 24 x_0_0*x_0_4 + 240 x_0_0*x_1_0
      + 840 x_0_0*x_2_0 + 360 x_0_0*x_3_0 + 120 x_0_0*x_4_0 - 960 x_0_0*y_0
      + 120 x_0_0*capacity_bin_0@int_slack@0
      + 240 x_0_0*capacity_bin_0@int_slack@1
      + 480 x_0_0*capacity_bin_0@int_slack@2
      + 120 x_0_0*capacity_bin_0@int_slack@3 + 312 x_0_1^2 + 24 x_0_1*x_0_2
      + 24 x_0_1*x_0_3 + 24 x_0_1*x_0_4 + 240 x_0_1*x_1_1 + 840 x_0_1*x_2_1
      + 360 x_0_1*x_3_1 + 120 x_0_1*x_4_1 - 960 x_0_1*y

In [42]:
cplex = CplexOptimizer()

result = cplex.solve(qubo)
print(result)

MissingOptionalLibraryError: "The 'CPLEX' library is required to use 'CplexOptimizer'. You can install it with 'pip install 'qiskit-optimization[cplex]''."

In [7]:
weights = [3, 2, 3, 1, 1]  # Weights of 4 items
C = 6  # Bin capacity
brute_force(weights, C)

NameError: name 'brute_force' is not defined